# Classification Notebook `iris-dataset`

## Add JupyterHelper and Download Dependencies

In [1]:
%jars ../libs/JupyterHelper-1.0-SNAPSHOT-jar-with-dependencies.jar

In [2]:
%%loadFromPOM

<properties>
    <deepnetts.version>1.12</deepnetts.version>
    <visrec.version>1.0.1</visrec.version>
</properties>

<dependency>
  <groupId>javax.visrec</groupId>
  <artifactId>visrec-api</artifactId>
  <version>1.0.1</version>
</dependency>
<dependency>
  <groupId>javax.visrec</groupId>
  <artifactId>visrec-ri</artifactId>
  <version>1.0.1</version>
  <type>jar</type>
  <exclusions>
      <exclusion>
          <groupId>com.deepnetts</groupId>
          <artifactId>deepnetts-core</artifactId>
      </exclusion>
  </exclusions>
</dependency>
<dependency>
    <groupId>com.deepnetts</groupId>
    <artifactId>deepnetts-core</artifactId>
    <version>${deepnetts.version}</version>
</dependency>

## Imports

In [7]:
import java.nio.file.*;

import java.io.ObjectInputStream;

import deepnetts.data.DataSets;
import deepnetts.eval.ClassifierEvaluator;
import deepnetts.eval.ConfusionMatrix;
import deepnetts.net.FeedForwardNetwork;
import deepnetts.net.layers.activation.ActivationType;
import deepnetts.net.loss.LossType;
import deepnetts.net.train.BackpropagationTrainer;
import deepnetts.net.train.opt.OptimizerType;
import deepnetts.util.DeepNettsException;
import deepnetts.util.FileIO;
import javax.visrec.ml.data.DataSet;
import javax.visrec.ml.eval.EvaluationMetrics;
import deepnetts.util.FileIO;

import ch.bytecrowd.JupyterHelper;

## Download Dataset

In [4]:
var IRIS_URL  = "https://raw.githubusercontent.com/deepnetts/deepnetts-communityedition/e8dbb4b35e8373720598b9d98d7755725dd52c53/deepnetts-examples/datasets/iris.csv";
var IRIS_FILE = "/tmp/iris.csv";

In [5]:
JupyterHelper.downloadFile(IRIS_URL, IRIS_FILE);
System.out.println("[+] Dataset saved to: " + IRIS_FILE);

EvalException: java.net.UnknownHostException: raw.githubusercontent.com

## Prepare DataSet

In [23]:
var numInputs = 4;  // corresponds to number of input features
var numOutputs = 3; // corresponds to number of possible classes/categories
var test_train_ratio = 0.6;

System.out.println("[+] Loading the data in to a 'deepnetts.data.DataSet'");
DataSet dataSet = DataSets.readCsv(IRIS_FILE, numInputs, numOutputs, true);

[+] Loading the data in to a 'deepnetts.data.DataSet'


## Splitting Dataset (Train/Test)

In [24]:
System.out.println("[+] Splitting datasets");
DataSet[] trainTestSet = dataSet.split(test_train_ratio, 1 - test_train_ratio);
System.out.println(String.format("[+] Training data size = %d rows", trainTestSet[0].size()));
System.out.println(String.format("[+] Testing data size  = %d rows", trainTestSet[1].size()));

[+] Splitting datasets
[+] Training data size = 90 rows
[+] Testing data size  = 60 rows


## Create a Neuronal Network

In [25]:
System.out.println("[+] Creating the Neural Network");
// create instance of multi addLayer percetpron using builder
FeedForwardNetwork neuralNet = FeedForwardNetwork.builder()
        .addInputLayer(numInputs)
        .addFullyConnectedLayer(5, ActivationType.TANH)
        .addOutputLayer(numOutputs, ActivationType.SOFTMAX)
        .lossFunction(LossType.CROSS_ENTROPY)
        .randomSeed(456)
        .build();
System.out.println(neuralNet.toString());

[+] Creating the Neural Network
Input Layer { width:4, height:1, depth:1 }
Fully Connected Layer { width:5 activation:TANH}
Output Layer { width:3, activation:SOFTMAX}



## Train the Neuronal Network

In [26]:
BackpropagationTrainer trainer = neuralNet.getTrainer();
trainer.setMaxError(0.04f);
trainer.setLearningRate(0.01f);
trainer.setMomentum(0.9f);
trainer.setOptimizer(OptimizerType.MOMENTUM);

neuralNet.train(trainTestSet[0]);

------------------------------------------------------------------------------------------------------------------------------------------------
TRAINING NEURAL NETWORK
------------------------------------------------------------------------------------------------------------------------------------------------
Epoch:1, Time:0ms, TrainError:0.8266282, TrainErrorChange:0.8266282, TrainAccuracy: 0.484375
Epoch:2, Time:0ms, TrainError:0.53173405, TrainErrorChange:-0.29489416, TrainAccuracy: 0.484375
Epoch:3, Time:0ms, TrainError:0.51558506, TrainErrorChange:-0.016148984, TrainAccuracy: 0.48958334
Epoch:4, Time:0ms, TrainError:0.5127226, TrainErrorChange:-0.0028624535, TrainAccuracy: 0.82539684
Epoch:5, Time:0ms, TrainError:0.5108404, TrainErrorChange:-0.0018821955, TrainAccuracy: 0.48958334
Epoch:6, Time:0ms, TrainError:0.5342339, TrainErrorChange:0.023393512, TrainAccuracy: 0.46682099
Epoch:7, Time:0ms, TrainError:0.47011894, TrainErrorChange:-0.06411499, TrainAccuracy: 0.52941173
Epoch

Epoch:81, Time:0ms, TrainError:0.55288893, TrainErrorChange:-1.5258789E-5, TrainAccuracy: 0.48958334
Epoch:82, Time:0ms, TrainError:0.5528741, TrainErrorChange:-1.4841557E-5, TrainAccuracy: 0.48958334
Epoch:83, Time:0ms, TrainError:0.5528598, TrainErrorChange:-1.4305115E-5, TrainAccuracy: 0.48958334
Epoch:84, Time:0ms, TrainError:0.55284554, TrainErrorChange:-1.424551E-5, TrainAccuracy: 0.48958334
Epoch:85, Time:0ms, TrainError:0.5528316, TrainErrorChange:-1.3947487E-5, TrainAccuracy: 0.48958334
Epoch:86, Time:0ms, TrainError:0.55281794, TrainErrorChange:-1.3649464E-5, TrainAccuracy: 0.48958334
Epoch:87, Time:0ms, TrainError:0.5528043, TrainErrorChange:-1.3649464E-5, TrainAccuracy: 0.48958334
Epoch:88, Time:0ms, TrainError:0.55279124, TrainErrorChange:-1.3053417E-5, TrainAccuracy: 0.48958334
Epoch:89, Time:0ms, TrainError:0.5527779, TrainErrorChange:-1.335144E-5, TrainAccuracy: 0.48958334
Epoch:90, Time:0ms, TrainError:0.55276495, TrainErrorChange:-1.2934208E-5, TrainAccuracy: 0.489583

Epoch:163, Time:0ms, TrainError:0.5523596, TrainErrorChange:-3.3974648E-6, TrainAccuracy: 0.48958334
Epoch:164, Time:0ms, TrainError:0.55235595, TrainErrorChange:-3.6358833E-6, TrainAccuracy: 0.48958334
Epoch:165, Time:0ms, TrainError:0.5523525, TrainErrorChange:-3.4570694E-6, TrainAccuracy: 0.48958334
Epoch:166, Time:0ms, TrainError:0.5523491, TrainErrorChange:-3.3974648E-6, TrainAccuracy: 0.48958334
Epoch:167, Time:0ms, TrainError:0.5523456, TrainErrorChange:-3.516674E-6, TrainAccuracy: 0.48958334
Epoch:168, Time:0ms, TrainError:0.55234194, TrainErrorChange:-3.6358833E-6, TrainAccuracy: 0.48958334
Epoch:169, Time:0ms, TrainError:0.5523383, TrainErrorChange:-3.6358833E-6, TrainAccuracy: 0.48958334
Epoch:170, Time:0ms, TrainError:0.55233467, TrainErrorChange:-3.6358833E-6, TrainAccuracy: 0.48958334
Epoch:171, Time:0ms, TrainError:0.552331, TrainErrorChange:-3.695488E-6, TrainAccuracy: 0.48958334
Epoch:172, Time:0ms, TrainError:0.55232704, TrainErrorChange:-3.9339066E-6, TrainAccuracy: 

Epoch:245, Time:0ms, TrainError:0.55217606, TrainErrorChange:-2.9802322E-6, TrainAccuracy: 0.48958334
Epoch:246, Time:0ms, TrainError:0.552173, TrainErrorChange:-3.0398369E-6, TrainAccuracy: 0.48958334
Epoch:247, Time:0ms, TrainError:0.5521693, TrainErrorChange:-3.695488E-6, TrainAccuracy: 0.48958334
Epoch:248, Time:0ms, TrainError:0.5521654, TrainErrorChange:-3.9339066E-6, TrainAccuracy: 0.48958334
Epoch:249, Time:0ms, TrainError:0.552161, TrainErrorChange:-4.4107437E-6, TrainAccuracy: 0.48958334
Epoch:250, Time:0ms, TrainError:0.55215615, TrainErrorChange:-4.827976E-6, TrainAccuracy: 0.48958334
Epoch:251, Time:0ms, TrainError:0.5521502, TrainErrorChange:-5.9604645E-6, TrainAccuracy: 0.48958334
Epoch:252, Time:0ms, TrainError:0.55214334, TrainErrorChange:-6.854534E-6, TrainAccuracy: 0.48958334
Epoch:253, Time:0ms, TrainError:0.55213493, TrainErrorChange:-8.404255E-6, TrainAccuracy: 0.48958334
Epoch:254, Time:1ms, TrainError:0.55212456, TrainErrorChange:-1.0371208E-5, TrainAccuracy: 0.

Epoch:327, Time:1ms, TrainError:0.5521534, TrainErrorChange:-5.9604645E-7, TrainAccuracy: 0.48958334
Epoch:328, Time:0ms, TrainError:0.5521527, TrainErrorChange:-7.1525574E-7, TrainAccuracy: 0.48958334
Epoch:329, Time:0ms, TrainError:0.55215204, TrainErrorChange:-6.556511E-7, TrainAccuracy: 0.48958334
Epoch:330, Time:0ms, TrainError:0.5521515, TrainErrorChange:-5.364418E-7, TrainAccuracy: 0.48958334
Epoch:331, Time:0ms, TrainError:0.5521507, TrainErrorChange:-7.748604E-7, TrainAccuracy: 0.48958334
Epoch:332, Time:0ms, TrainError:0.5521502, TrainErrorChange:-5.364418E-7, TrainAccuracy: 0.48958334
Epoch:333, Time:0ms, TrainError:0.5521496, TrainErrorChange:-5.9604645E-7, TrainAccuracy: 0.48958334
Epoch:334, Time:0ms, TrainError:0.55214906, TrainErrorChange:-5.364418E-7, TrainAccuracy: 0.48958334
Epoch:335, Time:0ms, TrainError:0.5521484, TrainErrorChange:-6.556511E-7, TrainAccuracy: 0.48958334
Epoch:336, Time:0ms, TrainError:0.55214775, TrainErrorChange:-6.556511E-7, TrainAccuracy: 0.489

...

Epoch:99496, Time:0ms, TrainError:0.55194885, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99497, Time:0ms, TrainError:0.55194885, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99498, Time:0ms, TrainError:0.55194885, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99499, Time:1ms, TrainError:0.55194885, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99500, Time:0ms, TrainError:0.55194885, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99501, Time:0ms, TrainError:0.55194885, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99502, Time:0ms, TrainError:0.55194885, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99503, Time:0ms, TrainError:0.55194885, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99504, Time:0ms, TrainError:0.55194885, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99505, Time:0ms, TrainError:0.55194885, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99506, Time:1ms, TrainError:0.55194885, TrainErrorChan

Epoch:99583, Time:0ms, TrainError:0.55194885, TrainErrorChange:5.9604645E-8, TrainAccuracy: 0.48958334
Epoch:99584, Time:0ms, TrainError:0.55194885, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99585, Time:1ms, TrainError:0.55194885, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99586, Time:0ms, TrainError:0.5519488, TrainErrorChange:-5.9604645E-8, TrainAccuracy: 0.48958334
Epoch:99587, Time:0ms, TrainError:0.5519488, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99588, Time:0ms, TrainError:0.5519488, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99589, Time:0ms, TrainError:0.55194885, TrainErrorChange:5.9604645E-8, TrainAccuracy: 0.48958334
Epoch:99590, Time:0ms, TrainError:0.5519488, TrainErrorChange:-5.9604645E-8, TrainAccuracy: 0.48958334
Epoch:99591, Time:0ms, TrainError:0.5519488, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99592, Time:0ms, TrainError:0.5519488, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99593, Time:0ms, Train

Epoch:99668, Time:0ms, TrainError:0.55194885, TrainErrorChange:5.9604645E-8, TrainAccuracy: 0.48958334
Epoch:99669, Time:0ms, TrainError:0.55194885, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99670, Time:1ms, TrainError:0.55194885, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99671, Time:0ms, TrainError:0.55194885, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99672, Time:0ms, TrainError:0.55194885, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99673, Time:0ms, TrainError:0.55194885, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99674, Time:0ms, TrainError:0.55194885, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99675, Time:0ms, TrainError:0.55194885, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99676, Time:0ms, TrainError:0.55194885, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99677, Time:1ms, TrainError:0.55194885, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99678, Time:0ms, TrainError:0.55194885, Train

Epoch:99753, Time:0ms, TrainError:0.5519488, TrainErrorChange:-5.9604645E-8, TrainAccuracy: 0.48958334
Epoch:99754, Time:0ms, TrainError:0.5519488, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99755, Time:0ms, TrainError:0.55194885, TrainErrorChange:5.9604645E-8, TrainAccuracy: 0.48958334
Epoch:99756, Time:0ms, TrainError:0.55194885, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99757, Time:0ms, TrainError:0.55194885, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99758, Time:0ms, TrainError:0.55194885, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99759, Time:0ms, TrainError:0.55194885, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99760, Time:0ms, TrainError:0.55194885, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99761, Time:0ms, TrainError:0.5519488, TrainErrorChange:-5.9604645E-8, TrainAccuracy: 0.48958334
Epoch:99762, Time:0ms, TrainError:0.55194885, TrainErrorChange:5.9604645E-8, TrainAccuracy: 0.48958334
Epoch:99763, Time:0ms, Tr

Epoch:99840, Time:0ms, TrainError:0.55194885, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99841, Time:0ms, TrainError:0.55194885, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99842, Time:0ms, TrainError:0.55194885, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99843, Time:0ms, TrainError:0.55194885, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99844, Time:0ms, TrainError:0.55194885, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99845, Time:0ms, TrainError:0.55194885, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99846, Time:0ms, TrainError:0.55194885, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99847, Time:0ms, TrainError:0.55194885, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99848, Time:0ms, TrainError:0.55194885, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99849, Time:0ms, TrainError:0.55194885, TrainErrorChange:0.0, TrainAccuracy: 0.48958334
Epoch:99850, Time:0ms, TrainError:0.55194885, TrainErrorChan

## Evaluating the model

In [28]:
System.out.println("[+] Evaluating the model");
ClassifierEvaluator evaluator = new ClassifierEvaluator();
EvaluationMetrics em = evaluator.evaluate(neuralNet, trainTestSet[1]);

[+] Evaluating the model


In [29]:
System.out.println("[+] CLASSIFIER EVALUATION METRICS");
System.out.println(em);
System.out.println("[+] CONFUSION MATRIX");
ConfusionMatrix cm = evaluator.getConfusionMatrix();
System.out.println(cm);

[+] CLASSIFIER EVALUATION METRICS
Accuracy: 0.5185185 (How often is classifier correct in total)
Precision: NaN (How often is classifier correct when it gives positive prediction)
F1Score: NaN (Average of precision and recall)
Recall: NaN (When it is actually positive class, how often does it give positive prediction)

[+] CONFUSION MATRIX
                none    setosaversicolor virginica
      none         0         0         0         0
    setosa         0        24         0         0
versicolor         0         0        20         0
 virginica         0         0        16         0



## Save the model

In [6]:
var EXPORT_PATH = "/tmp/IrisClassifier.dnet";
System.out.println("[+] Saving the model to: " + EXPORT_PATH);
FileIO.writeToFile(neuralNet, EXPORT_PATH);

[+] Saving the model to: /tmp/IrisClassifier.dnet


CompilationException: 

## Import the model

In [35]:
System.out.println("[+] Importing the model from: " + EXPORT_PATH);
FeedForwardNetwork neuralNet;
try (ObjectInputStream ois = new ObjectInputStream(new FileInputStream(EXPORT_PATH))) {
    neuralNet = FeedForwardNetwork.class.cast(ois.readObject()) ;
}

[+] Importing the model from: /tmp/IrisClassifier.dnet
